In [ ]:
import os
import re
import urllib.request
import json
import time
from bs4 import BeautifulSoup

In [ ]:
def create_base():
    directory = 'Республика Башкортостан'
    if not os.path.exists(directory):
        os.makedirs(directory)
    table = directory + os.sep + 'metatab.csv'
    header = 'path\tauthor\theader\tcreated\t' \
            'sphere\ttopic\tstyle\taudience_age\t' \
            'audience_level\taudience_size\tsource\t' \
            'publication\tpubl_year\tmedium\t' \
            'country\tregion\tlanguage\n'
    with open(table, 'w', encoding = 'utf-8') as csv:
        csv.write(header)
    return directory, table

In [ ]:
def downloaden(link):
    time.sleep(1)
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    req = urllib.request.Request(link, headers={'User-Agent': user_agent})
    with urllib.request.urlopen(req) as resp:
        try:
            html = resp.read().decode('utf-8')
        except UnicodeDecodeError:
            html = resp.read().decode('utf-16')
    return html

In [ ]:
def find_archive(directory):
    hm_page = 'https://resbash.ru'
    hm_page_html = downloaden(hm_page)
    pattern = '<a href="([^"]+)">Архив газеты'
    arch_page = re.search(pattern, hm_page_html).group(1)
    arch_url = hm_page + arch_page
    arch_html = downloaden(arch_url)
    pattern = 'Архив газеты!(.*?)\n'
    arch_links = re.search(pattern, arch_html).group(1)
    arch_list = re.findall("href='[^']+'", arch_links)
    links_list = []
    for link in arch_list:
        link = re.sub("href=", '', link)
        link = re.sub("'", '', link)
        link = hm_page + link
        links_list.append(link)
    return links_list

In [ ]:
def every_month(fold_link):
    splt = fold_link.split('/')
    path = splt[len(splt)-1] + os.sep + splt[len(splt)-2]
    html = downloaden(fold_link)
    pattern = "<a href='[^']+'[^>]*>№"
    href_list = re.findall(pattern, html)
    issues_links = []
    for href in href_list:
        link = re.search("href='([^']+)'", href).group(1)
        lnk = r"https://resbash.ru" + link
        issues_links.append(lnk)
    return path, issues_links

In [ ]:
def make_dir_tree(path, directory, root):
    p_dir = directory + os.sep + root + os.sep + path
    if not os.path.exists(p_dir):
        os.makedirs(p_dir)

In [ ]:
def every_issue(issue_link):
    html = downloaden(issue_link)
    pattern = "<h2><a href='[^']+'"
    stat_raw = re.findall(pattern, html)
    stat_list = []
    for href in stat_raw:
        link = re.search("href='([^']+)'", href).group(1)
        link = r'https://resbash.ru' + link
        stat_list.append(link)
    return stat_list

In [ ]:
def get_info(html):
    author_p = '>Автор:&nbsp;([А-Яа-яЁё -]+)[,<]'
    if re.search(author_p, html, re.DOTALL):
        author = re.search(author_p, html, re.DOTALL).group(1)
    else:
        author = 'None'
    created_p = "Опубликовано: (\d\d\.\d\d.\d\d\d\d)"
    if re.search(created_p, html, re.DOTALL):
        created = re.search(created_p, html, re.DOTALL).group(1)
        p_year = created[len(created)-4:len(created)]
    else:
        created = 'None'
        p_year = 'None'
    topic_p = '<div class="div_anons_news">Статьи рубрики <a[^>]+>([^<]+)<'
    if re.search(topic_p, html, re.DOTALL):
        topic = re.search(topic_p, html, re.DOTALL).group(1)
    else:
        topic = 'None'
    return author, created, p_year, topic

In [ ]:
def txt_art(directory, html, path, aut, cre, top, s_link):
    tit_patt1 = '<title>[^-] - ([^<]*)<'
    tit_patt2 = '<title>([^<]*)<'
    s = os.sep
    c = 0
    if re.search(tit_patt1, html, re.DOTALL):
        title = re.search(tit_patt1, html, re.DOTALL).group(1)
        title = re.sub('[^А-Яа-яЁёCc]', '', title)
        file_path = directory + s + 'plain' + s + path + s + title + '.txt'
    elif re.search(tit_patt2, html, re.DOTALL):
        title = re.search(tit_patt2, html, re.DOTALL).group(1)
        title = re.sub('[^А-Яа-яЁёCc]', '', title)
        file_path = directory + s + 'plain' + s + path + s + title + '.txt'
    else:
        title = 'None'
        c += 1
        ps_t = title + str(c)
        file_path = directory + s + 'plain' + s + path + s + ps_t + '.txt'
    m1 = '@au ' + aut
    m2 = '@ti ' + title
    m3 = '@da ' + cre
    m4 = '@topic ' + top
    m5 = '@url ' + s_link
    meta = [m1, m2, m3, m4, m5]    
    with open(file_path, 'w', encoding='utf-8') as art_file:
        art_file.write('\n'.join(meta))
        art_file.write('\n')
    x_sub = "<div style='padding:10px 20px;font-size:13px;'>"
    y_sub = "</div"
    x = html.find(x_sub)
    y = html.find(y_sub, x)
    art_text = html[x+len(x_sub):y]
    soup = BeautifulSoup(art_text, 'html.parser')
    with open(file_path, 'a', encoding='utf-8') as art_file:
        art_file.write(soup.get_text())
    return title, file_path

In [ ]:
def update_table(table, m_tuple):
    row = '%s\t%s\t%s\t%s\tпублицистика\t%s\t' \
        'нейтральный\tн-возраст\tн-уровень\tреспубл' \
        'иканская\t%s\tРеспублика Башкортостан\t%s' \
        '\tгазета\tРоссия\tБашкортостан\tru\n'
    with open(table, 'a', encoding='utf-8') as m_tab:
        m_tab.write(row % m_tuple)

In [ ]:
def article_main(s_link, table, d, path):
    html = downloaden(s_link)
    author, created, p_year, topic = get_info(html)
    title, f_path = txt_art(d, html, path, author, created, topic, s_link)
    m_tuple = (f_path, author, title, created, topic, s_link, p_year)
    update_table(table, m_tuple)

In [ ]:
def crawling(directory, table):
    month_list = find_archive(directory)
    if len(month_list):
        for mont_link in month_list:
            path, iss_list = every_month(mont_link)
            make_dir_tree(path, directory, 'plain')
            for iss_link in iss_list:
                stat_list = every_issue(iss_link)
                if len(stat_list) > 0:
                    for s_link in stat_list:
                        article_main(s_link, table, directory, path)

In [ ]:
def plain_search(directory):
    root = directory + os.sep + 'plain'
    months_list = []
    year_list = os.listdir(path=root)
    for year in year_list:
        y = root + os.sep + year
        m_list = os.listdir(path=y)
        for m in m_list:
            month_path = y + os.sep + m
            months_list.append(month_path)
    return months_list

файл mystem.exe я положил в ту же папку, что и файл с кодом

In [ ]:
def fold_visit(list_of_dirs):
    for path in list_of_dirs:
        p_xml = re.sub('plain', 'mystem-xml', path)
        p_pl = re.sub('plain', 'mystem-plain', path)
        if not os.path.exists(p_xml):
            os.makedirs(p_xml)
        if not os.path.exists(p_pl):
            os.makedirs(p_pl)
        s_pl = "mystem.exe -cligd --eng-gr " 
        s_xml = s_pl + '--format xml '
        for fl in os.listdir(path):
            inp_name = path + os.sep + fl
            f = open(inp_name, 'r', encoding='utf-8')
            info = f.read()
            f.close()
            f2 = open('input.txt', 'w', encoding='utf-8')
            f2.write(info)
            f2.close()
            txt_command = s_pl + ' input.txt output.txt'
            os.system(txt_command)
            with open('output.txt', 'r', encoding='utf-8') as f:
                mstxt = f.read()
            name = p_pl + os.sep + fl
            with open(name, 'w', encoding='utf-8') as f:
                f.write(mstxt)
            xml_command = s_xml + ' input.txt output.xml'
            os.system(xml_command)
            with open('output.xml', 'r', encoding='utf-8') as f:
                msxml = f.read()
            flx = re.sub('txt', 'xml', fl)
            name = p_xml + os.sep + flx
            with open(name, 'w', encoding='utf-8') as f:
                f.write(msxml)

In [ ]:
def main():
    directory, table = create_base()
    crawling(directory, table)
    list_of_dirs = plain_search(directory)
    fold_visit(list_of_dirs)

In [ ]:
if __name__ == '__main__':
    main()
    